# Análisis de Imágenes con DeepForest
Este notebook procesa imágenes en tres resoluciones para detección de árboles usando DeepForest, ajusta un umbral de confianza basado en ejemplos de la documentación, calcula métricas de evaluación (precisión, recall, F1) y guarda los resultados junto con las imágenes anotadas.

## 1. Clonar repositorio y configuración inicial

In [1]:
import os
import git

# Parámetros de repositorio y carpetas
REPO_URL = 'https://github.com/maraosoc/citrus3-detector.git'
REPO_DIR = 'citrus3-detector'
DATA_DIR = os.path.join(REPO_DIR, 'data', 'samples')
RESULT_DIR = 'resultados_citrus'

# Clonar si no existe
if not os.path.exists(REPO_DIR):
    print('Clonando repositorio...')
    git.Repo.clone_from(REPO_URL, REPO_DIR)
else:
    print('El repositorio ya existe localmente.')


El repositorio ya existe localmente.


## 2. Configuración de modelo y umbrales

In [2]:
from deepforest import main

# Cargar modelo pre-entrenado
modelo = main.deepforest()
modelo.use_release()

# Ajuste de umbral de confianza según ejemplos de la documentación (score_thresh = 0.3)
modelo.config['score_thresh'] = 0.3  # Retener predicciones con score >= 0.3


Reading config file: d:\MAESTRIA_CIENCIAS_DE_LOS_DATOS\SEMESTRE 2025-I\bosqueprof\Lib\site-packages\deepforest\data\deepforest_config.yml


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Reading config file: d:\MAESTRIA_CIENCIAS_DE_LOS_DATOS\SEMESTRE 2025-I\bosqueprof\Lib\site-packages\deepforest\data\deepforest_config.yml


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## 3. Funciones auxiliares: IoU, emparejamiento y métricas

In [3]:
import numpy as np

def compute_iou(box1, box2):
    """Calcula el IoU entre dos cajas [xmin, ymin, xmax, ymax]."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0

def match_boxes(predictions, ground_truth, iou_threshold=0.4):
    """Empareja predicciones y ground truth según IoU y devuelve TP, FP, FN."""
    matched_gt = set()
    tp = 0
    for _, pred in predictions.iterrows():
        pred_box = [pred.xmin, pred.ymin, pred.xmax, pred.ymax]
        for idx, true in ground_truth.iterrows():
            if idx in matched_gt:
                continue
            true_box = [true.xmin, true.ymin, true.xmax, true.ymax]
            if compute_iou(pred_box, true_box) >= iou_threshold:
                tp += 1
                matched_gt.add(idx)
                break
    fp = len(predictions) - tp
    fn = len(ground_truth) - tp
    return tp, fp, fn

def compute_metrics(tp, fp, fn):
    """Calcula precisión, recall y F1."""
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1


## 4. Procesamiento de imágenes y evaluación

In [4]:
import cv2
import pandas as pd
from deepforest.visualize import plot_predictions

# Resoluciones a comparar
RESOLUCIONES = {
    'original': None,  # None = mantener tamaño original
    'medio': (1033, 939),
    'pequeno': (516, 469)
}

# Conteos aproximados si no hay anotaciones
approx_counts = {'single_batch_2': 110}

os.makedirs(RESULT_DIR, exist_ok=True)
resumen_general = []

for img_file in os.listdir(DATA_DIR):
    if not img_file.lower().endswith(('.tif', '.png', '.jpg')):
        continue
    img_path = os.path.join(DATA_DIR, img_file)
    basename = os.path.splitext(img_file)[0]
    img = cv2.imread(img_path)
    if img is None:
        continue
    # Cargar anotaciones ground truth si existen
    gt_path = os.path.join(REPO_DIR, 'data', 'annotations', f'{basename}_gt.csv')
    if os.path.exists(gt_path):
        ground_truth = pd.read_csv(gt_path)
    else:
        ground_truth = None

    for res_name, size in RESOLUCIONES.items():
        if size:
            img_resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
        else:
            img_resized = img.copy()

        # Predicción
        boxes = modelo.predict_image(image=img_resized, return_plot=False)
        # Filtrar por score_thresh
        boxes = boxes[boxes.score >= modelo.config['score_thresh']]

        # Evaluación
        if ground_truth is not None:
            tp, fp, fn = match_boxes(boxes, ground_truth)
            precision, recall, f1 = compute_metrics(tp, fp, fn)
        else:
            tp = fp = fn = None
            detected = len(boxes)
            true_count = approx_counts.get(basename, None)
            err_abs = detected - true_count if true_count else None
            precision = recall = f1 = None

        # Guardar CSV de predicciones
        boxes.to_csv(os.path.join(RESULT_DIR, f'cajas_{res_name}_{basename}.csv'), index=False)
        # Guardar imagen anotada
        annotated = plot_predictions(img_resized, boxes)
        cv2.imwrite(os.path.join(RESULT_DIR, f'annotated_{res_name}_{basename}.png'), annotated)

        # Agregar al resumen
        resumen_general.append({
            'Imagen': basename,
            'Resolución': res_name,
            'Detections': len(boxes),
            'TP': tp, 'FP': fp, 'FN': fn,
            'Precision': precision, 'Recall': recall, 'F1': f1
        })

# Guardar resumen general
df_resumen = pd.DataFrame(resumen_general)
df_resumen.to_csv(os.path.join(RESULT_DIR, 'resumen_general.csv'), index=False)


## 5. Resultados
- Los CSV y las imágenes anotadas se guardan en `resultados_citrus`.
- El resumen general con métricas está en `resumen_general.csv`.